# **RAG with LLaMa 13B**

In this notebook we'll explore how we can use the open source Llama-13b-chat model in both Hugging Face transformers and LangChain.

Installing the required libraries

In [1]:
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.9 MB/s

 Initializing the embedding pipeline that will handle the transformation of our docs into vector embeddings.
 Using the sentence-transformers/all-MiniLM-L6-v2 model for embedding.

In [2]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# New Section

Using our embedding model to create document embedding

In [3]:
docs = [
    "this is one document",
    "and another document"
]

embeddings = embed_model.embed_documents(docs)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 2 doc embeddings, each with a dimensionality of 384.


Building the Vector Database:
We will be using the pinecone vector index for our RAG pipeline.
Using the embedding pipeline to build our embeddings and store them in a Pinecone vector index. Using my Pinecone API key to initialize the index.

In [4]:
import os
import pinecone
from google.colab import userdata

# API key from app.pinecone.io and environment from console
# The secrets are stored in colab secrets. It is covered in the following article
# https://colab.research.google.com/drive/1DJ5wW7-gIozuuhFnqZ494u2tLQDvNWVS#scrollTo=Q571xCe3CBXU&line=6&uniqifier=1
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = userdata.get('PINECONE_ENVIRONMENT')

pinecone.init(
    api_key=os.environ.get(PINECONE_API_KEY) or PINECONE_API_KEY,
    environment=os.environ.get(PINECONE_ENVIRONMENT) or PINECONE_ENVIRONMENT
)

Initializing the database

In [5]:
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

Creating the vector database

In [6]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.04843,
 'namespaces': {'': {'vector_count': 4843}},
 'total_vector_count': 4843}

We will use a set of Arxiv papers related to (and including) the Llama 2 research paper as our dataset.

In [7]:
from datasets import load_dataset

data = load_dataset(
    'jamescalam/llama-2-arxiv-papers-chunked',
    split='train'
)
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})

Loading the dataset to the vector database.

In [8]:
data = data.to_pandas()

batch_size = 32

for i in range(0, len(data), batch_size):
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    texts = [x['chunk'] for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

Checking the stats of the vector database after adding the dataset

In [9]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.04843,
 'namespaces': {'': {'vector_count': 4843}},
 'total_vector_count': 4843}

 To initialize a text-generation pipeline with Hugging Face transformers, initializing the model and move it to CUDA-enabled GPU

In [10]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# The HF auth token is stored in colab serects and get it from there.
HF_AUTH =  userdata.get('HF_AUTH')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=HF_AUTH
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=HF_AUTH
)
model.eval()
print(f"Model loaded on {device}")

(…)a-2-13b-chat-hf/resolve/main/config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


Tokenizing the plain text to LLM readable token IDs.

In [11]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=HF_AUTH
)

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-13b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Initializing the Hugging Face pipeline

In [12]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

Checking the text generation

In [13]:
res = generate_text("What is Llama 2")
print(res[0]["generated_text"])

What is Llama 2.0?

Llama 2.0 is a new version of the popular open-source vulnerability scanner and web application security testing tool, OWASP ZAP (Zed Attack Proxy). It was released in December 2019 and includes several new features and improvements over the previous version. Some of the key changes in Llama 2.0 include:

1. Improved performance: Llama 2.0 is faster and more efficient than its predecessor, with improved performance and reduced memory usage.
2. Enhanced user interface: The new version has a modernized user interface that is easier to use and navigate, with improved layout and design.
3. New features: Llama 2.0 includes several new features, such as support for testing WebSocket applications, improved handling of HTTP/2 requests, and better integration with other tools and plugins.
4. Better compatibility: Llama 2.0 is compatible with a wider range of operating systems and platforms, including Windows, macOS, and Linux.
5. Enhanced reporting: The new version includes 

In [ ]:
res = generate_text("what is so special about llama 2?")
print(res[0]["generated_text"])

what is so special about llama 2?

Answer: Llama 2 is a unique and special animal for several reasons. Here are some of the most notable features that make it stand out:

1. Size: Llamas are known for their size, and Llama 2 is no exception. It is one of the largest llamas in existence, with some individuals reaching heights of over 6 feet (1.8 meters) at the shoulder and weighing up to 400 pounds (180 kilograms).
2. Coat: Llama 2 has a distinctive coat that is soft, fine, and silky to the touch. The coat can be a variety of colors, including white, cream, beige, and brown.
3. Temperament: Llama 2 is known for its friendly and docile nature. They are social animals that thrive on human interaction and are often used as therapy animals due to their calm demeanor.
4. Intelligence: Llama 2 is highly intelligent and can learn a wide range of tasks, from simple commands like "sit" and "stay" to more complex tasks like pulling carts or carrying packs.
5. Adaptability: Llama 2 is highly adapt

Integrating with langchain to connect the vector database

In [14]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [15]:
llm(prompt="What is Llama 2")

'.0?\n\nLlama 2.0 is a new version of the popular open-source vulnerability scanner and web application security testing tool, OWASP ZAP (Zed Attack Proxy). It was released in December 2019 and includes several new features and improvements over the previous version. Some of the key changes in Llama 2.0 include:\n\n1. Improved performance: Llama 2.0 is faster and more efficient than its predecessor, with improved performance and reduced memory usage.\n2. Enhanced user interface: The new version has a modernized user interface that is easier to use and navigate, with improved layout and design.\n3. New features: Llama 2.0 includes several new features, such as support for testing WebSocket applications, improved handling of HTTP/2 requests, and better integration with other tools and plugins.\n4. Better compatibility: Llama 2.0 is compatible with a wider range of operating systems and platforms, including Windows, macOS, and Linux.\n5. Enhanced reporting: The new version includes improv

In [ ]:
llm(prompt="what is so special about llama 2?")

'\n\nAnswer: Llama 2 is a unique and special animal for several reasons. Here are some of the most notable features that make it stand out:\n\n1. Size: Llamas are known for their size, and Llama 2 is no exception. It is one of the largest llamas in existence, with some individuals reaching heights of over 6 feet (1.8 meters) at the shoulder and weighing up to 400 pounds (180 kilograms).\n2. Coat: Llama 2 has a distinctive coat that is soft, fine, and silky to the touch. The coat can be a variety of colors, including white, cream, beige, and brown.\n3. Temperament: Llama 2 is known for its friendly and docile nature. They are social animals that thrive on human interaction and are often used as therapy animals due to their calm demeanor.\n4. Intelligence: Llama 2 is highly intelligent and can learn a wide range of tasks, from simple commands like "sit" and "stay" to more complex tasks like pulling carts or carrying packs.\n5. Adaptability: Llama 2 is highly adaptable and can survive in 

Initializing the retrieval QA chain for the RAG pipeline

In [16]:
from langchain.vectorstores import Pinecone

text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

Checking the similarity search with the results of the top 3 searches

In [17]:
query = 'what makes llama 2 special?'

vectorstore.similarity_search(
    query,  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='Ricardo Lopez-Barquilla, Marc Shedroﬀ, Kelly Michelena, Allie Feinstein, Amit Sangani, Geeta\nChauhan,ChesterHu,CharltonGholson,AnjaKomlenovic,EissaJamil,BrandonSpence,Azadeh\nYazdan, Elisa Garcia Anzano, and Natascha Parks.\n•ChrisMarra,ChayaNayak,JacquelinePan,GeorgeOrlin,EdwardDowling,EstebanArcaute,Philomena Lobo, Eleonora Presani, and Logan Kerr, who provided helpful product and technical organization support.\n46\n•Armand Joulin, Edouard Grave, Guillaume Lample, and Timothee Lacroix, members of the original\nLlama team who helped get this work started.\n•Drew Hamlin, Chantal Mora, and Aran Mun, who gave us some design input on the ﬁgures in the\npaper.\n•Vijai Mohan for the discussions about RLHF that inspired our Figure 20, and his contribution to the\ninternal demo.\n•Earlyreviewersofthispaper,whohelpedusimproveitsquality,includingMikeLewis,JoellePineau,\nLaurens van der Maaten, Jason Weston, and Omer Levy.', metadata={'source': 'http://arxiv.org/pdf/230

Adding the vector database to the LLM for the RAG pipeline

In [18]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

LLM without RAG

In [ ]:
llm('what is so special about llama 2?')

'\n\nAnswer: Llama 2 is a unique and special animal for several reasons. Here are some of the most notable features that make it stand out:\n\n1. Size: Llamas are known for their size, and Llama 2 is no exception. It is one of the largest llamas in existence, with some individuals reaching heights of over 6 feet (1.8 meters) at the shoulder and weighing up to 400 pounds (180 kilograms).\n2. Coat: Llama 2 has a distinctive coat that is soft, fine, and silky to the touch. The coat can be a variety of colors, including white, cream, beige, and brown.\n3. Temperament: Llama 2 is known for its friendly and docile nature. They are social animals that thrive on human interaction and are often used as therapy animals due to their calm demeanor.\n4. Intelligence: Llama 2 is highly intelligent and can learn a wide range of tasks, from simple commands like "sit" and "stay" to more complex tasks like pulling carts or carrying packs.\n5. Adaptability: Llama 2 is highly adaptable and can survive in 

 **LLM With RAG**

Without RAG the LLM talks about the animal Llama, with RAG the LLM explains about the pretrained and fine tuned LLMs

In [19]:
rag_pipeline('what is so special about llama 2?')

{'query': 'what is so special about llama 2?',
 'result': ' Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) developed and released by GenAI, Meta. The models are optimized for dialogue use cases and outperform open-source chat models on most benchmarks tested. Additionally, they are considered a suitable substitute for closed-source models like ChatGPT, BARD, and Claude.\n\nPlease let me know if you need any further information or clarification.'}

Similarity search before ingesting new dataset gives random response about the query.

In [19]:
vectorstore.similarity_search(
    "See Metric Name format for more details",  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='vehicle-mounted camera. Each image has a resolution of\nMethodCityscapes Test Set Cityscapes Val Set\nAP AP 50% AP 100m AP 50m AP muCov\nvan den Brand et al. [29] 2.3% 3.7% 3.9% 4.9% - Cordts et al. [6] 4.6% 12.9% 7.7% 10.3% - Uhrig et al. [28] 8.9% 21.1% 15.3% 16.7% 9.9% Ours 19.4% 35.3% 31.4% 36.8% 21.2% 68.0%\nTable 1: Cityscapes instance segmentation results using metrics deﬁned in [6] for AP and [26] for muCov.\nMethod Person Rider Car Truck Bus Train Motorcycle Bicycle\nvan den Brand et al. [29] - - 18.2% - - - - Cordts et al. [6] 1.3% 0.6% 10.5% 6.1% 9.7% 5.9% 1.7% 0.5%\nUhrig et al. [28] 12.5% 11.7% 22.5% 3.3% 5.9% 3.2% 6.9% 5.1%', metadata={'source': 'http://arxiv.org/pdf/1611.08303', 'title': 'Deep Watershed Transform for Instance Segmentation'}),
 Document(page_content='Figure G.46: Formatted dataset example for Arithmetic 3DContext!Q: What is 556 plus 497?\nA:\nTarget Completion !1053\nFigure G.47: Formatted dataset example for Arithmetic 3D+\nContex

The text files are retrieved from GitHub as a new dataset.

In [20]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!git clone https://github.com/MahaJayapal/LLM.git
dataset = '/content/LLM/dataset'

Cloning into 'LLM'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 15), reused 26 (delta 6), pack-reused 0
Receiving objects: 100% (43/43), 89.83 KiB | 1.60 MiB/s, done.
Resolving deltas: 100% (15/15), done.


Ingesting the new data to the vector database.

In [21]:
def ingest_data(data_dir):
  for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        # Read text file
        with open(os.path.join(data_dir, filename), "r") as f:
            text = f.read()
        metadata = {'filename': filename, 'Release-notes': 'week-of-september-29-2023'}
        vectorstore.add_texts([text], [metadata])


In [22]:
ingest_data(dataset)

Upserted vectors:   0%|          | 0/1 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/1 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/1 [00:00<?, ?it/s]

Making a similarity search to check our ingestion.

In [24]:
vectorstore.similarity_search(
    "See Metric Name format for more details",  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='Metrics Engine Enhancements\nMetric Name length limit increased\nThe metric name size has been increased to 255 characters, and you can now utilize up to 10 underscores within this limit. Previously, only 100 characters were permitted with a maximum of 5 underscores.\n\nWith this extended limit, you can now create more descriptive metric names that accommodate various makes, models, and version details.\n\nSee Metric Name format for more details.', metadata={'Release-notes': 'week-of-september-29-2023', 'filename': 'Metrics Engine Enhancements.txt'}),
 Document(page_content='vehicle-mounted camera. Each image has a resolution of\nMethodCityscapes Test Set Cityscapes Val Set\nAP AP 50% AP 100m AP 50m AP muCov\nvan den Brand et al. [29] 2.3% 3.7% 3.9% 4.9% - Cordts et al. [6] 4.6% 12.9% 7.7% 10.3% - Uhrig et al. [28] 8.9% 21.1% 15.3% 16.7% 9.9% Ours 19.4% 35.3% 31.4% 36.8% 21.2% 68.0%\nTable 1: Cityscapes instance segmentation results using metrics deﬁned in [6] f

The output from RAG for the prompt, after ingesting new data to the content store is more appropriate.

In [25]:
rag_pipeline('See Metric Name format for more details')

{'query': 'See Metric Name format for more details',
 'result': ' The metric name size has been increased to 255 characters, and you can now utilize up to 10 underscores within this limit. Previously, only 100 characters were permitted with a maximum of 5 underscores.'}